In [2]:
import os
import json
import datetime
from isatools.model import (Investigation, Study, Assay, Person, Material,
                            DataFile, plink,
                            OntologySource, OntologyAnnotation, Sample,
                            Source, Characteristic, Protocol, Process)
from isatools import isatab
from isatools.isajson import ISAJSONEncoder

final_dir = os.path.abspath(os.path.join('output2', 'issue-brapi'))

In [3]:
investigation = Investigation()
investigation.identifier = "BRAPI-test-unused-source"
investigation.title = "BRAPI-test-unused-source"
investigation.description = "this is test to understand the conditions under which ISA-API will serialize or not serialize a Source entity declared but not used in a workflow. Note: while the python ISA-API does not serialize in the Tab format, the information is available from ISA-JSON."

In [4]:
prs_test_study = Study(filename="s_prs_test.txt")

prs_test_study.identifier = "PRS"
prs_test_study.title = "Unused Sources"
prs_test_study.description = "testing if the python ISA-API supports unusued Sources in ISA-Tab serialization"



In [5]:
prs = Person(last_name="Rocca-Serra", first_name="Philippe", mid_initials="T", affiliation="OeRC", email="prs@hotmail.com" )
prs_test_study.contacts.append(prs)
print(prs.mid_initials)

T


In [6]:
ncbi_taxon = OntologySource(name='NCBITaxon', description="NCBI Taxonomy")
human_characteristic= Characteristic(category=OntologyAnnotation(term="Organism"),
                                     value=OntologyAnnotation(term="Homo Sapiens", term_source=ncbi_taxon,
                                                              term_accession="http://purl.bioontology.org/ontology/NCBITAXON/9606"))

## Creating ISA Sources

In [7]:
subject_0 = Source(name='human individual-0', characteristics=[human_characteristic]) 
subject_1 = Source(name='human individual-1', characteristics=[human_characteristic]) 
subject_2 = Source(name='human individual-2', characteristics=[human_characteristic]) 



## Creating ISA Samples

In [8]:
sample_0 = Sample(name='SBJ0_sample1')
# note that 2 samples are generated from subject_1
sample_1 = Sample(name='SBJ1_sample1')
sample_2 = Sample(name='SBJ1_sample2')
# note that no sample is generated from subject_
sample_3 = Sample(name='SBJ2')

## Associating Sources and Samples to the ISA.Study. object

In [9]:
prs_test_study.sources.append(subject_0)
prs_test_study.sources.append(subject_1)
prs_test_study.sources.append(subject_2)

prs_test_study.samples.append(sample_0)
prs_test_study.samples.append(sample_1)
prs_test_study.samples.append(sample_2)
#prs_test_study.samples.append(subject_2)
prs_test_study.samples.append(sample_3)

## Declaring Protocol objects

### Don't forget the protocol_type should be declared as an ISA Ontology Annotation

In [10]:
prs_protocol = Protocol(name="sample collection",
                             protocol_type=OntologyAnnotation(term="sample collection"))

### Adding the newly created protocol to the list of protocols associated with an ISA Study Object

In [11]:
prs_test_study.protocols.append(prs_protocol)

## Creating the ProtocolApplication events connecting Parent to Children Materials

### executing a protocol minimally or maximally by specifying date and performer of execution

In [12]:
prs_process0 = Process(executes_protocol=prs_protocol)
now = str(datetime.datetime.now().strftime("%Y-%m-%d"))
prs_process1 = Process(executes_protocol=prs_protocol, performer=prs.first_name, date_=now)
prs_process2 = Process(executes_protocol=prs_protocol, performer=prs.first_name, date_=now)
prs_process3 = Process(executes_protocol=prs_protocol, performer=prs.first_name, date_=now)

### Setting input and outputs of each sample collection protocol application

In [13]:
prs_process0.inputs.append(subject_0)
prs_process0.outputs.append(sample_0)
prs_process1.inputs.append(subject_1)
prs_process1.outputs.append(sample_1)
prs_process2.inputs.append(subject_1)
prs_process2.outputs.append(sample_2)

### Now the following tests if setting a protocol application with no output jinxes the ISA-API

In [14]:
prs_process3.inputs.append(subject_2)
prs_process3.outputs.append(sample_3)

### Here, the ISA Study object is updated by associating all the processes/protocol_applications to the process_sequence attribute.


In [15]:
prs_test_study.process_sequence.append(prs_process0)
prs_test_study.process_sequence.append(prs_process1)
prs_test_study.process_sequence.append(prs_process2)
prs_test_study.process_sequence.append(prs_process3)

## Creating an ISA Assay object - This is to test associating an ISA Source as input to an Assay

### Step1 - Create the ISA Assay Object

In [16]:
assay_on_source = Assay(measurement_type=OntologyAnnotation(term="phenotyping"),
                     technology_type=OntologyAnnotation(term=""),
                     filename="a_assay-test.txt")

### Step2 - Create a new ISA Protocol the type of which is `data acquisition`

In [17]:
assay_protocol = Protocol(name="assay-on-source",
                          protocol_type="data acquisition")



### Step3 - Remember to add the new protocol to the ISA.Study object

In [18]:
prs_test_study.protocols.append(assay_protocol)

### Step4 - Create the Protocol Application event which generates an ISA DataFile

In [19]:
assay_process = Process(executes_protocol=assay_protocol, performer=prs.first_name)

### Step5 - Create an ISA DataFile object

In [20]:
dummy_file= DataFile(filename="dummy.txt")

### Step6 - Set ProtocolApplication/Process inputs and outputs testing if an ISA.Source can be used in an ISA.Assay

In [21]:
assay_process.inputs.append(sample_0)
assay_process.outputs.append(dummy_file)

### Step7 - Associate the newly created ISA.DataFile with the ISA.Assay object

In [22]:
assay_on_source.data_files.append(dummy_file)

### Step8 - Link and Connect ISA.ProtocolApplications via the process_sequence attribute

In [23]:
assay_on_source.process_sequence.append(prs_process3)
assay_on_source.process_sequence.append(assay_process)

### Step9 - Update the ISA.Assay Object by listing all ISA.Materials used in the Assay associated ProtocolApplications

In [24]:
assay_on_source.samples.append(sample_3)
#assay_on_source.other_material.append(subject_2)
plink(prs_process3, assay_process)

In [25]:
prs_test_study.process_sequence.append(assay_process)

### Step10 - Update the ISA.Study Object by adding the ISA Assay object to the ISA.Study assays attribute

In [26]:
prs_test_study.assays.append(assay_on_source)

In [27]:
investigation.studies.append(prs_test_study)

In [28]:
dataframes = isatab.dump_tables_to_dataframes(investigation)

In [29]:
dataframes['s_prs_test.txt']

,Source Name,Characteristics[Organism],Term Source REF,Term Accession Number,Protocol REF,Date,Performer,Sample Name
0,human individual-1,Homo Sapiens,NCBITaxon,http://purl.bioontology.org/ontology/NCBITAXON...,sample collection,2020-12-09,Philippe,SBJ1_sample1
1,human individual-1,Homo Sapiens,NCBITaxon,http://purl.bioontology.org/ontology/NCBITAXON...,sample collection,2020-12-09,Philippe,SBJ1_sample2
2,human individual-2,Homo Sapiens,NCBITaxon,http://purl.bioontology.org/ontology/NCBITAXON...,sample collection,2020-12-09,Philippe,SBJ2


In [30]:
#dataframes['assay-test.txt']

In [31]:
isatab.dump(isa_obj=investigation, output_path=final_dir)

isatools.model.Investigation(identifier='BRAPI-test-unused-source', filename='', title='BRAPI-test-unused-source', submission_date='', public_release_date='', ontology_source_references=[], publications=[], contacts=[], studies=[isatools.model.Study(filename='s_prs_test.txt', identifier='PRS', title='Unused Sources', description='testing if the python ISA-API supports unusued Sources in ISA-Tab serialization', submission_date='', public_release_date='', contacts=[isatools.model.Person(last_name='Rocca-Serra', first_name='Philippe', mid_initials='T', email='prs@hotmail.com', phone='', fax='', address='', affiliation='OeRC', roles=[], comments=[])], design_descriptors=[], publications=[], factors=[], protocols=[isatools.model.Protocol(name='sample collection', protocol_type=isatools.model.OntologyAnnotation(term='sample collection', term_source=None, term_accession='', comments=[]), uri='', version='', parameters=[], components=[], comments=[]), isatools.model.Protocol(name='assay-on-sou

In [32]:
isa_j = json.dumps(investigation, cls=ISAJSONEncoder, sort_keys=True, indent=4, separators=(',', ': '))
open("isa_as_json_from_dumps1.json","w").write(isa_j) # this call write the string 'isa_j' to the file called 'isa_as_json_from_dumps.json'

14349

In [33]:
test_dir = os.path.abspath(os.path.join('output2', 'issue-brapi-unused-source'))

In [34]:
my_json_report = isatab.validate(open(os.path.join(test_dir, 'i_investigation.txt')))

2020-12-09 21:45:00,630 [INFO]: isatab.py(validate:4202) >> Loading... /Users/philippe/Documents/git/isa-create-notebook/notebooks/output2/issue-brapi-unused-source/i_investigation.txt
2020-12-09 21:45:00,785 [INFO]: isatab.py(validate:4204) >> Running prechecks...
2020-12-09 21:45:00,845 [INFO]: isatab.py(validate:4225) >> Finished prechecks...
2020-12-09 21:45:00,846 [INFO]: isatab.py(validate:4226) >> Loading configurations found in /Users/philippe/.pyenv/versions/3.8.0/envs/isa-api-py38/src/isatools/isatools/resources/config/xml
2020-12-09 21:45:00,923 [INFO]: isatab.py(validate:4231) >> Using configurations found in /Users/philippe/.pyenv/versions/3.8.0/envs/isa-api-py38/src/isatools/isatools/resources/config/xml
2020-12-09 21:45:00,924 [INFO]: isatab.py(validate:4233) >> Checking investigation file against configuration...
2020-12-09 21:45:00,926 [INFO]: isatab.py(validate:4236) >> Finished checking investigation file
2020-12-09 21:45:00,927 [INFO]: isatab.py(validate:4255) >> Lo

In [35]:
print(my_json_report)

{'errors': [], 'warnings': [], 'info': [{'message': 'Found 2 study groups in s_prs_test.txt', 'supplemental': 'Found 2 study groups in s_prs_test.txt', 'code': 5001}, {'message': 'Found -1 study groups in a_assay_test.txt', 'supplemental': 'Found -1 study groups in a_assay_test.txt', 'code': 5001}], 'validation_finished': True}
